In [1]:
# Note: The files read by this script are generated by get_data.py

import os
import psycopg2
import warnings
import pandas as pd
import datetime as dt
from dotenv import load_dotenv

load_dotenv()

warnings.filterwarnings('ignore')

In [2]:
data_dir = "data"  # change if needed
all_data = []

def extract_symbol_from_filename(filename):
    return filename.replace("_history.csv", "").replace("_", ".")

def transform_csv(file_path, symbol):
    # Read multi-row header CSV
    df = pd.read_csv(file_path, header=[0, 1])

    # Flatten column names (use top-level or fallback to 'Date')
    df.columns = [col[0] if col[0] != "Price" else "Date" for col in df.columns]

    # Drop the top two rows (Ticker/Date & symbol row)
    df = df.drop(index=[0, 1]).reset_index(drop=True)

    # Add extracted symbol to a new column
    df["Symbol"] = symbol

    # Convert date and numeric fields
    df["Date"] = pd.to_datetime(df["Date"], errors="coerce")
    for col in ["Open", "High", "Low", "Close", "Volume"]:
        df[col] = pd.to_numeric(df[col], errors="coerce")

    # Keep only the required columns
    df = df[["Date", "Symbol", "Open", "High", "Low", "Close", "Volume"]]

    return df

In [3]:
for file in os.listdir(data_dir):
    if file.endswith("_history.csv"):
        path = os.path.join(data_dir, file)
        symbol = extract_symbol_from_filename(file)
        try:
            transformed_df = transform_csv(path, symbol)
            all_data.append(transformed_df)
            print(f"✅ Processed {file} → Symbol: {symbol}")
        except Exception as e:
            print(f"⚠️ Failed to process {file}: {e}")

✅ Processed ACC_NS_history.csv → Symbol: ACC.NS
✅ Processed BANKBARODA_NS_history.csv → Symbol: BANKBARODA.NS
✅ Processed FEDERALBNK_NS_history.csv → Symbol: FEDERALBNK.NS
✅ Processed IRFC_NS_history.csv → Symbol: IRFC.NS
✅ Processed LTTS_NS_history.csv → Symbol: LTTS.NS
✅ Processed LT_NS_history.csv → Symbol: LT.NS
✅ Processed NTPCGREEN_NS_history.csv → Symbol: NTPCGREEN.NS
✅ Processed NTPC_NS_history.csv → Symbol: NTPC.NS
✅ Processed RCF_NS_history.csv → Symbol: RCF.NS
✅ Processed RELIANCE_NS_history.csv → Symbol: RELIANCE.NS
✅ Processed STARCEMENT_NS_history.csv → Symbol: STARCEMENT.NS
✅ Processed TATAPOWER_NS_history.csv → Symbol: TATAPOWER.NS


In [ ]:
# if all_data:
#     final_df = pd.concat(all_data, ignore_index=True)
# else:
#     print("⚠️ No data to combine.")

# final_df

🧾 Combined DataFrame Preview:
        Date  Symbol         Open         High          Low        Close  \
0 2025-02-19  ACC.NS  1851.000000  1905.150024  1846.150024  1867.099976   
1 2025-02-20  ACC.NS  1867.650024  1890.000000  1854.550049  1886.849976   
2 2025-02-21  ACC.NS  1885.000000  1900.000000  1867.699951  1884.000000   
3 2025-02-24  ACC.NS  1881.000000  1886.449951  1845.050049  1851.099976   
4 2025-02-25  ACC.NS  1857.000000  1860.000000  1825.050049  1834.050049   

     Volume  
0  187021.0  
1  134186.0  
2  231565.0  
3  176866.0  
4  156142.0  


In [ ]:
if all_data:
    final_df = pd.concat(all_data, ignore_index=True)
    
    # Define output path
    output_path = os.path.join(data_dir, "combined_stock_data.csv")
    
    # Save to CSV
    final_df.to_csv(output_path, index=False)
    print(f"✅ Combined data saved to {output_path}")
else:
    print("⚠️ No data to combine.")

✅ Combined data saved to data\combined_stock_data.csv
